In [3]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
import keras
from keras.layers import Dense
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential

%matplotlib inline

In [4]:
train_csv = tf.keras.utils.get_file("train.csv", "https://storage.googleapis.com/tf-datasets/titanic/train.csv")
test_csv = tf.keras.utils.get_file("eval.csv", "https://storage.googleapis.com/tf-datasets/titanic/eval.csv")

In [5]:
train_df = pd.read_csv(train_csv)


In [6]:
train_df.head(10)


,survived,sex,age,n_siblings_spouses,parch,fare,class,deck,embark_town,alone
0,0,male,22.0,1,0,7.2500,Third,unknown,Southampton,n
1,1,female,38.0,1,0,71.2833,First,C,Cherbourg,n
2,1,female,26.0,0,0,7.9250,Third,unknown,Southampton,y
3,1,female,35.0,1,0,53.1000,First,C,Southampton,n
4,0,male,28.0,0,0,8.4583,Third,unknown,Queenstown,y
5,0,male,2.0,3,1,21.0750,Third,unknown,Southampton,n
6,1,female,27.0,0,2,11.1333,Third,unknown,Southampton,n
7,1,female,14.0,1,0,30.0708,Second,unknown,Cherbourg,n
8,1,female,4.0,1,1,16.7000,Third,G,Southampton,n
9,0,male,20.0,0,0,8.0500,Third,unknown,Southampton,y


In [7]:
test_df = pd.read_csv(test_csv)

In [8]:
test_df.head(10)

,survived,sex,age,n_siblings_spouses,parch,fare,class,deck,embark_town,alone
0,0,male,35.0,0,0,8.0500,Third,unknown,Southampton,y
1,0,male,54.0,0,0,51.8625,First,E,Southampton,y
2,1,female,58.0,0,0,26.5500,First,C,Southampton,y
3,1,female,55.0,0,0,16.0000,Second,unknown,Southampton,y
4,1,male,34.0,0,0,13.0000,Second,D,Southampton,y
5,1,female,15.0,0,0,8.0292,Third,unknown,Queenstown,y
6,0,female,8.0,3,1,21.0750,Third,unknown,Southampton,n
7,0,male,21.0,0,0,8.0500,Third,unknown,Southampton,y
8,0,female,18.0,2,0,18.0000,Third,unknown,Southampton,n
9,1,female,19.0,0,0,7.8792,Third,unknown,Queenstown,y


In [9]:
train_df.isnull().sum()

survived              0
sex                   0
age                   0
n_siblings_spouses    0
parch                 0
fare                  0
class                 0
deck                  0
embark_town           0
alone                 0
dtype: int64

In [10]:
test_df.isnull().sum()

survived              0
sex                   0
age                   0
n_siblings_spouses    0
parch                 0
fare                  0
class                 0
deck                  0
embark_town           0
alone                 0
dtype: int64

In [11]:
train_df.columns

Index(['survived', 'sex', 'age', 'n_siblings_spouses', 'parch', 'fare',
       'class', 'deck', 'embark_town', 'alone'],
      dtype='object')

In [12]:
encoder = LabelEncoder()

In [13]:
train_df.iloc[:, 5] = encoder.fit_transform(train_df.iloc[:, 5])
train_df.iloc[:, 6] = encoder.fit_transform(train_df.iloc[:, 6])
train_df.iloc[:, 7] = encoder.fit_transform(train_df.iloc[:, 7])
train_df.iloc[:, 8] = encoder.fit_transform(train_df.iloc[:, 8])
train_df.iloc[:, 9] = encoder.fit_transform(train_df.iloc[:, 9])
train_df.iloc[:, 1] = encoder.fit_transform(train_df.iloc[:, 1])

In [14]:
test_df.iloc[:, 5] = encoder.fit_transform(test_df.iloc[:, 5])
test_df.iloc[:, 6] = encoder.fit_transform(test_df.iloc[:, 6])
test_df.iloc[:, 7] = encoder.fit_transform(test_df.iloc[:, 7])
test_df.iloc[:, 8] = encoder.fit_transform(test_df.iloc[:, 8])
test_df.iloc[:, 1] = encoder.fit_transform(test_df.iloc[:, 1])

In [15]:
test_df_new = test_df.iloc[:, test_df.columns != 'alone']

In [16]:
X = train_df.iloc[:, train_df.columns != 'alone']

In [17]:
y = np.array(train_df['alone'])

In [18]:
y = y.reshape(-1,1)

In [19]:
X.head()

,survived,sex,age,n_siblings_spouses,parch,fare,class,deck,embark_town
0,0,1,22.0,1,0,16,2,7,2
1,1,0,38.0,1,0,178,0,2,0
2,1,0,26.0,0,0,34,2,7,2
3,1,0,35.0,1,0,161,0,2,2
4,0,1,28.0,0,0,41,2,7,1


In [20]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.35)

In [21]:
X_train.shape

(407, 9)

In [22]:
X_test.shape

(220, 9)

In [23]:
scaler = StandardScaler()

In [24]:
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [25]:
model = Sequential()

# Enter input layer
model.add(Dense(50, input_shape = (9,), name = 'Input'))

# Enter hidden layer 1
model.add(Dense(100, activation = 'relu', name = "Dense_1"))

# Enter hidden layer 2
model.add(Dense(100, activation = 'relu', name = 'Dense_2'))

# Enter output layer
model.add(Dense(20, activation = 'softmax', name = 'Output'))

In [26]:
# Compilation of the sequential model

model.compile(loss = keras.losses.SparseCategoricalCrossentropy(), 
              optimizer = keras.optimizers.Adam(learning_rate=0.01), 
              metrics = [keras.metrics.SparseCategoricalAccuracy()])

In [27]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Input (Dense)                (None, 50)                500       
_________________________________________________________________
Dense_1 (Dense)              (None, 100)               5100      
_________________________________________________________________
Dense_2 (Dense)              (None, 100)               10100     
_________________________________________________________________
Output (Dense)               (None, 20)                2020      
Total params: 17,720
Trainable params: 17,720
Non-trainable params: 0
_________________________________________________________________


In [28]:
model.fit(X_train, y_train, epochs = 10, batch_size = 64, verbose = 2)

Epoch 1/10
 - 0s - loss: 1.1904 - sparse_categorical_accuracy: 0.6855
Epoch 2/10
 - 0s - loss: 0.1252 - sparse_categorical_accuracy: 0.9509
Epoch 3/10
 - 0s - loss: 0.0153 - sparse_categorical_accuracy: 0.9926
Epoch 4/10
 - 0s - loss: 0.0032 - sparse_categorical_accuracy: 0.9975
Epoch 5/10
 - 0s - loss: 9.7822e-04 - sparse_categorical_accuracy: 1.0000
Epoch 6/10
 - 0s - loss: 5.2390e-04 - sparse_categorical_accuracy: 1.0000
Epoch 7/10
 - 0s - loss: 1.4511e-05 - sparse_categorical_accuracy: 1.0000
Epoch 8/10
 - 0s - loss: 2.4731e-05 - sparse_categorical_accuracy: 1.0000
Epoch 9/10
 - 0s - loss: 3.9020e-05 - sparse_categorical_accuracy: 1.0000
Epoch 10/10
 - 0s - loss: 1.6594e-05 - sparse_categorical_accuracy: 1.0000


In [29]:
model.evaluate(X_test, y_test)

220/220 [==============================] - 0s 135us/step


[4.510712884227401e-06, 1.0]

In [30]:
model.evaluate(X_train, y_train)

407/407 [==============================] - 0s 38us/step


[1.0417113646608034e-05, 1.0]

In [31]:
y_pred = model.predict(X_test)

In [44]:
print("X=%s, Predicted=%s" % (X_test[0], y_test[0]))

X=[-0.80484364  0.77206116  1.31006291 -0.50475351 -0.48621218  1.61925455
 -1.57985269 -2.46199706 -1.94680434], Predicted=[1]
